# SpamDetector

## Preprocessing

In [2]:
%run ExtractContent.py TR/ TRAIN/

The destination directory is newly created.


In [2]:
# Get labels
import string
import csv
labels = list()
with open('spam-mail.tr.label','rb') as f :
    labels_reader = csv.reader(f, delimiter=',')
    count_row = 0

    for row in labels_reader:
        if count_row != 0 :
            labels.append(row[1])
        count_row = count_row + 1
#print labels

In [3]:
# Get mails
from sklearn.feature_extraction.text import TfidfVectorizer
import codecs

mails = list()
for i in range(1, 2501) :
    with codecs.open('TRAIN/TRAIN_%s.eml'%i, 'r', encoding='utf-8', errors='ignore') as f :
        mail = f.read()
        mails.append(mail)

# Compute TD-IDF features
vectorizer = TfidfVectorizer(stop_words='english', strip_accents='ascii')
mail_vectors = vectorizer.fit_transform(mails)

In [4]:
# Divise dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(mail_vectors, labels, test_size=0.33, random_state=42)


## Machine Learning

### Supervised - Decision tree

In [5]:
# Train model
print "Training..."
from sklearn import tree

clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2) # classifier = learning algorithm that will learn from the training dataset
model = clf.fit(X_train, y_train) # finds patterns in training data and create a set of rules (the model)
print "Model trained!"

training_error = 1-clf.score(X_train, y_train)
test_error = 1-clf.score(X_test, y_test)
print "Training error : " + str(training_error)
print "Test error : " + str(test_error)



Training...
Model trained!
Training error : 0.151641791045
Test error : 0.139393939394


### Supervised - SVM

In [6]:
print "Some code"
from sklearn import svm
#X = [[0, 0], [1, 1]]
#y = [0, 1]
clf = svm.SVC()
clf.fit(X_train, y_train)

training_error = 1-clf.score(X_train, y_train)
test_error = 1-clf.score(X_test, y_test)
print "Training error : " + str(training_error)
print "Test error : " + str(test_error)


Some code
Training error : 0.314626865672
Test error : 0.305454545455
